<a href="https://colab.research.google.com/github/mostafa-asefy/neural-networks-from-zero/blob/main/Artificial_Neural_Networks_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Class Definition

In [16]:
import numpy as np


class NeuralNetwork:

    # initialize the neural network 
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # set number of nodes in input, hidden and output layers
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # set the learning rate
        self.learning_rate = learning_rate

        # link weight matrices, w_input_hidden and w_hidden_output
        # weights inside the arrays are w_i_j, where link is from node 
        # i to node j in the next layer
        # w11 w21
        # w12 w22 etc
        self.w_i_h = np.random.rand(self.hidden_nodes, self.input_nodes) - 0.5
        self.w_h_o = np.random.rand(self.output_nodes, self.hidden_nodes) - 0.5
        pass

    # train the neural network
    def train():
        pass

    # query the neural network
    def query():
        pass

# Neural Network Instance Creation

In [17]:
nn = NeuralNetwork(3, 3, 3, 0.3)
print(nn.w_i_h)

[[ 0.08022783  0.32007052 -0.44493165]
 [-0.15973388 -0.499542   -0.24130081]
 [ 0.13126193 -0.48995979 -0.39662463]]
